In [2]:
from data_structures import PatientData
from sklearn.base import BaseEstimator
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from typing import Tuple
from sklearn.model_selection import train_test_split
import numpy as np
from dataclasses import dataclass
from typing import Dict, List
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from decoders import DecodingResult, ConceptDecoder, ConceptPairDataset, DecodingResultsManager


In [3]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress convergence warnings from scikit-learn
warnings.filterwarnings("ignore", category=ConvergenceWarning)


todo: imp dropout - add firing rate filter method to patient data to return neurons above threshold, implement by area stuff


combined patients -- cant i call pseudo/normal datasets indepdently then hstack after? onset dims are same so it should work

In [4]:
p566 = PatientData(pid='566')
p563 = PatientData(pid='563')
p562 = PatientData(pid='562')

./Data/40m_act_24_S06E01_30fps_character_frames.csv
./Data/40m_act_24_S06E01_30fps_character_frames.csv
./Data/40m_act_24_S06E01_30fps_character_frames.csv


In [5]:
THRESHOLD = 0.1


p566_fr_neurons = p566.filter_neurons_by_fr(neurons=p566.neurons, window=(p566.times_dict['movie_start_rel'], p566.times_dict['preSleep_recall_start_rel']), threshold=THRESHOLD)
p566_mtl_fr_neurons = p566.filter_mtl_neurons(neurons=p566_fr_neurons)

p563_fr_neurons = p563.filter_neurons_by_fr(neurons=p563.neurons, window=(p563.times_dict['movie_start_rel'], p563.times_dict['preSleep_recall_start_rel']), threshold=THRESHOLD)
p563_mtl_fr_neurons = p563.filter_mtl_neurons(neurons=p563_fr_neurons)

p562_fr_neurons = p562.filter_neurons_by_fr(neurons=p562.neurons, window=(p562.times_dict['movie_start_rel'], p562.times_dict['preSleep_recall_start_rel']), threshold=THRESHOLD)
p562_mtl_fr_neurons = p562.filter_mtl_neurons(neurons=p562_fr_neurons)

In [6]:

print(len(p562_fr_neurons))
print(len(p562_mtl_fr_neurons))
print()
print(len(p563_fr_neurons))
print(len(p563_mtl_fr_neurons))
print()
print(len(p566_fr_neurons))
print(len(p566_mtl_fr_neurons))

89
3

111
64

75
18


In [7]:
concept_pairs_to_decode = []
concept_list = list(p566.movie_df.columns)
#remove non-concept columns if they exist
concept_list = [col for col in concept_list if col not in ['Frame', 'time_sec', 'rel_corrected_time_sec', 'uncorrected_time_sec']]


# doing to 5 for time

for i, concept1 in enumerate(concept_list[:]):
    for concept2 in concept_list[i+1:]: #avoid duplicates and self-pairs
        concept_pairs_to_decode.append((concept1, concept2))

print(f"Number of concept pairs to decode: {len(concept_pairs_to_decode)}")
print(concept_pairs_to_decode[:5]) # Print first 5 pairs as example

Number of concept pairs to decode: 136
[('A.Amar', 'A.Fayed'), ('A.Amar', 'B.Buchanan'), ('A.Amar', 'C.Manning'), ('A.Amar', 'C.OBrian'), ('A.Amar', 'J.Bauer')]


In [ ]:
pseudo_manager562 = DecodingResultsManager(
    patient_data=p562,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons=p562_mtl_fr_neurons    # no kwargs -> default
)

normal_manager562 = DecodingResultsManager(
    patient_data=p562,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p562_mtl_fr_neurons)

In [ ]:
pseudo_manager563 = DecodingResultsManager(
    patient_data=p563,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons=p563_mtl_fr_neurons
    # no kwargs -> default
)

normal_manager563 = DecodingResultsManager(
    patient_data=p563,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p563_mtl_fr_neurons)

In [ ]:
pseudo_manager566 = DecodingResultsManager(
    patient_data=p566,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons=p566_mtl_fr_neurons
    # no kwargs -> default
)

normal_manager566 = DecodingResultsManager(
    patient_data=p566,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p566_mtl_fr_neurons)

todo: 

combine patients

8 best concepts

single patient normal + pseudopops

In [ ]:
for title, manager in {'normal_manager566': normal_manager566, 'normal_manager563': normal_manager563, 'normal_manager562': normal_manager562,
                       'pseudo_manager566': pseudo_manager566, 'pseudo_manager563': pseudo_manager563, 'pseudo_manager562': pseudo_manager562}.items():
    manager.run_decoding_for_pairs(num_iter=5)
    manager.plot_train_test_performance_heatmap(metric='test_accuracy')
    plt.savefig(f"{title}_test_accuracy")
    manager.plot_train_test_performance_heatmap() # auc roc
    plt.savefig(f"{title}_roc_auc")



combined patients (with mtl)

testing that mtl neuron filtering actually works

In [13]:
concept_pairs = [('A.Amar', 'C.Manning')]

x = DecodingResultsManager(
    patient_data=p566,
    concept_pairs=concept_pairs,
    epoch='movie',
    standardize=False,
    pseudo=False,
    neurons=p566_mtl_fr_neurons)


In [14]:
x.run_decoding_for_pairs()

Decoding for 566: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it]


In [22]:
x.results[('A.Amar', 'C.Manning')][0].data['X_test'].shape

(15, 169)

shit, we're not using mtl neurons
